# Named Entity Recognition Pipeline

El pipeline toma una URL de un feed en formato RSS, obtiene el título y descripción de los artículos en el feed, detecta las NER con un modelo pre-entrenado, y las muestra ordenadas por frecuencia de aparición.

### Versiones
Probado con:
* Almond 0.6.0
* Ammonite 1.6.7
* Scala library version **2.11.12** -- Copyright 2002-2017, LAMP/EPFL
* Java 1.8.0_282

Para ver más información ir a (Help -> About Scala Kernel)

## 1. Obtener texto

### 1.1 Importar librerías

In [1]:
// Equivalent of adding dependencies to maven or sbt files
// For example, to add "org.scalaj" %% "scalaj-http" % "2.4.2" 
import $ivy.`org.scalaj::scalaj-http:2.4.2`
// "org.scala-lang.modules" %% "scala-xml" % "1.3.0"
import $ivy.`org.scala-lang.modules::scala-xml:1.3.0`
import $ivy.`org.json4s::json4s-jackson:3.4.0`

import $ivy.$                              
// "org.scala-lang.modules" %% "scala-xml" % "1.3.0"

import $ivy.$                                        

import $ivy.$                                 

In [2]:
import scalaj.http.{Http, HttpResponse}
import scala.xml.XML
import scala.collection.mutable.MutableList
//import org.json4s.JsonDSL._
import org.json4s._
import org.json4s.jackson.JsonMethods._
implicit val formats = DefaultFormats

import scalaj.http.{Http, HttpResponse}

import scala.xml.XML

import scala.collection.mutable.MutableList
//import org.json4s.JsonDSL._

import org.json4s._

import org.json4s.jackson.JsonMethods._

formats: DefaultFormats.type = org.json4s.DefaultFormats$@2bdce90

### 1.1 Obtener el texto del RSS Feed

Realizamos una consulta HTTP, que nos devuelve una instancia de HTTPResponse. Dentro del atributo `body` de la HTTPResponse, se encuentra el texto del feed en formato XML. Luego, se parsea el XML para extraer los campos `title` y `description`.

In [3]:
abstract class Parser {
    val emptyText: String
    
    // Get the content
    def openURL(url: String): String = {
        try {
            var response = Http(url)
            .timeout(connTimeoutMs = 2000, readTimeoutMs = 5000)
            .asString
            response.body
        } catch {
            case e : Throwable  => println("Error in HTTP response")
            emptyText 
        }
    }
    
    // Extract text 
    def processText(rawText: String): Seq[String] 
    
    // Read the text
    def readText(url: String): Seq[String] = {
        processText(openURL(url))
    }
}

class RSSParser() extends Parser {
    val emptyText = "<rss></rss>"
    
    def processText(rawText: String): Seq[String] = {
        // convert the `String` to a `scala.xml.Elem`
        val xml = XML.loadString(rawText)
        // Extract text from title and description
        (xml \\ "item").map { item =>
            ((item \ "title").text ++ " " ++ (item \ "description").text)
        }
    }
}

class RedditParser() extends Parser {
    val emptyText = "{}"
    
    def processText(rawText: String): Seq[String] = {
        val urlPattern = "(https?|ftp|file)://[-a-zA-Z0-9+&@#/%?=~_|!:,.;]*[-a-zA-Z0-9+&@#/%=~_|]"
        val titles = (parse(rawText) \ "data" \ "children" \ "data" \ "title" )
         .extract[List[String]]
        val selftexts = (parse(rawText) \ "data" \ "children" \ "data" \ "selftext" )
         .extract[List[String]]
        val result = titles.zip(selftexts).map{case (a,b) => a ++ " " ++b}
        result.map(text => text.replaceAll(urlPattern, " "))
    }
}



defined class Parser
defined class RSSParser
defined class RedditParser

In [4]:
class FeedService() {
    var urls: MutableList[(String, Parser)] = new MutableList[(String, Parser)]()
    
    def subscribe(urlTemplate: String, params: Seq[String], parser: Parser) = {
        if (params == Seq()){
            urls ++= Seq((urlTemplate, parser))
        } else {
            val result = params.map(x => urlTemplate.format(x)).map(y =>(y, parser))
            urls ++= result
        }
    }
    
    def getText(): Seq[String] = {
        urls.flatMap{case (a, b) => b.readText(a)}
    }
}

defined class FeedService

In [5]:
val rss_parser = new RSSParser
val url_rss = "https://www.chicagotribune.com/arcio/rss/category/sports/?query=display_date:[now-2d+TO+now]&sort=display_date:desc"
val rssText = rss_parser.readText(url_rss)

rss_parser: RSSParser = ammonite.$sess.cmd2$Helper$RSSParser@6ea69848
url_rss: String = "https://www.chicagotribune.com/arcio/rss/category/sports/?query=display_date:[now-2d+TO+now]&sort=display_date:desc"
rssText: Seq[String] = List(
  "As United Center, other sites wind down, Chicago shifting vaccination focus to local events As the city of Chicago prepares to wind down its biggest mass vaccination site, officials said Tuesday they are focused on a hyperlocal that includes dozens of pop-up events, vaccine incentives and home visits.",
  "Chicago Bears are optimistic about their cornerback competition, but they still are considering adding a veteran free-agent option such as Bashaud Breeland The Chicago Bears are optimistic about the competition they have at cornerback with Desmond Trufant and Kindle Vildor. But the team still is window shopping and considering a veteran option such as Bashaud Breeland.",
  "Illinois\u2019 home Big Ten football game vs. Maryland is moved to a Friday I

In [7]:
val reddit_parser = new RedditParser
val url_reddit = "https://www.reddit.com/r/Android/hot/.json?count=10"
val redditText = reddit_parser.readText(url_reddit)

reddit_parser: RedditParser = ammonite.$sess.cmd2$Helper$RedditParser@9c36188
url_reddit: String = "https://www.reddit.com/r/Android/hot/.json?count=10"
redditText: Seq[String] = List(
  """Moronic Monday (May 24 2021) - Your weekly questions thread! Note 1. Join us at /r/MoronicMondayAndroid, a sub serving as a repository for our retired weekly threads. Just pick any thread and Ctrl-F your way to wisdom! 

Note 2. Join our IRC, and Telegram chat-rooms! [Please see our wiki for instructions.]( )""",
  "Google Assistant will soon be able to power off your Android - 9to5Google ",
  "Arm Announces Mobile Armv9 CPU Microarchitectures: Cortex-X2, Cortex-A710 &amp; Cortex-A510 ",
  "Anker teases Nebula Android TV dongle for 2021 release - 9to5Google ",
  "VideoCardz: \"ARM announces Mali-G710, G610, G510 and G310 graphics processing units\" ",
  "Galaxy Upcycling: How Samsung Ruined Their Best Idea in Years ",
  """ShieldTV Best Emulators "Guide" This is just to save someone searching for th

In [8]:
val servicio = new FeedService
servicio.subscribe("https://www.chicagotribune.com/arcio/rss/category/%s/?query=display_date:[now-2d+TO+now]&sort=display_date:desc", List("sports", "business"), rss_parser)
servicio.subscribe("https://www.reddit.com/r/Android/hot/.json?count=10", List(), reddit_parser)

servicio: FeedService = ammonite.$sess.cmd3$Helper$FeedService@7d40e3c
res7_1: MutableList[(String, Parser)] = MutableList(
  (
    "https://www.chicagotribune.com/arcio/rss/category/sports/?query=display_date:[now-2d+TO+now]&sort=display_date:desc",
    ammonite.$sess.cmd2$Helper$RSSParser@6ea69848
  ),
  (
    "https://www.chicagotribune.com/arcio/rss/category/business/?query=display_date:[now-2d+TO+now]&sort=display_date:desc",
    ammonite.$sess.cmd2$Helper$RSSParser@6ea69848
  ),
  (
    "https://www.reddit.com/r/Android/hot/.json?count=10",
    ammonite.$sess.cmd2$Helper$RedditParser@9c36188
  )
)
res7_2: MutableList[(String, Parser)] = MutableList(
  (
    "https://www.chicagotribune.com/arcio/rss/category/sports/?query=display_date:[now-2d+TO+now]&sort=display_date:desc",
    ammonite.$sess.cmd2$Helper$RSSParser@6ea69848
  ),
  (
    "https://www.chicagotribune.com/arcio/rss/category/business/?query=display_date:[now-2d+TO+now]&sort=display_date:desc",
    ammonite.$sess.cmd2$H

In [21]:
servicio.getText()

res20: Seq[String] = MutableList(
  "Illinois is set to play a Friday game against Maryland this Big Ten football season Illinois will play on a Friday this season, hosting Maryland on Sept. 17, the Big Ten announced.",
  "Chicago Bull Zach LaVine sells Lakeview mansion for $3M Chicago Bulls guard Zach LaVine on May 21 sold his five-bedroom mansion in the city's Lakeview neighborhood for $3 million.",
  "Aaron Rodgers doesn\u2019t attend the 1st day of Green Bay Packers OTAs \u2014 and remains noncommittal about his future in ESPN interview Green Bay Packers quarterback Aaron Rodgers wasn\u2019t present for the first day of organized team activities Monday, and his future with the team remains uncertain.",
  "US travel warning for COVID-19 won\u2019t affect Olympians, says Japan The Japanese government Tuesday was quick to deny a U.S. warning for Americans to avoid traveling to Japan would have an impact on Olympians wanting to compete in the postponed Tokyo Games.",
  "Chicago Blackha

## 2. Detectar las entidades nombradas

### 2.1 Crear el modelo

El **modelo** es sólo la función `getNEs`, que recibe una lista de textos.
Para cada texto, se separa las palabras del texto usando los espacios, y considera que es una entidad nombrada si empieza con mayúscula.

Este código lista los signos de puntuación y algunas palabras comunes del inglés que se van a sacar del texto.

In [23]:
val STOPWORDS = Seq (
    "i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you",
    "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she",
    "her", "hers", "herself", "it", "its", "itself", "they", "them", "your",
    "their", "theirs", "themselves", "what", "which", "who", "whom",
    "this", "that", "these", "those", "am", "is", "are", "was", "were",
    "be", "been", "being", "have", "has", "had", "having", "do", "does",
    "did", "doing", "a", "an", "the", "and", "but", "if", "or",
    "because", "as", "until", "while", "of", "at", "by", "for", "with",
    "about", "against", "between", "into", "through", "during", "before",
    "after", "above", "below", "to", "from", "up", "down", "in", "out",
    "off", "over", "under", "again", "further", "then", "once", "here",
    "there", "when", "where", "why", "how", "all", "any", "both", "each",
    "few", "more", "most", "other", "some", "such", "no", "nor", "not",
    "only", "own", "same", "so", "than", "too", "very", "s", "t", "can",
    "will", "just", "don", "should", "now", "on",
    // Contractions without '
    "im", "ive", "id", "Youre", "youd", "youve",
    "hes", "hed", "shes", "shed", "itd", "were", "wed", "weve",
    "theyre", "theyd", "theyve",
    "shouldnt", "couldnt", "musnt", "cant", "wont",
    // Common uppercase words
    "hi", "hello"
)

STOPWORDS: Seq[String] = List(
  "i",
  "me",
  "my",
  "myself",
  "we",
  "our",
  "ours",
  "ourselves",
  "you",
  "yours",
  "yourself",
  "yourselves",
  "he",
  "him",
  "his",
  "himself",
  "she",
  "her",
  "hers",
  "herself",
  "it",
  "its",
  "itself",
  "they",
  "them",
  "your",
  "their",
  "theirs",
  "themselves",
  "what",
  "which",
  "who",
  "whom",
  "this",
  "that",
  "these",
  "those",
  "am",
...

In [24]:
class NERModel(STOPWORDS: Seq[String]) {
    
    val punctuationSymbols = ".,()!?;:'`´\n"
    val punctuationRegex = "\\" + punctuationSymbols.split("").mkString("|\\")
    
    // Extract Named Entitis
    def getNEsSingle(text: String): Seq[String] = {
      text.replaceAll(punctuationRegex, "").split(" ")
        .filter { word:String => word.length > 1 &&
                  Character.isUpperCase(word.charAt(0)) &&
                  !STOPWORDS.contains(word.toLowerCase) }.toSeq
    }
    def getNEs(textList: Seq[String]): Seq[Seq[String]] = {
        textList.map(getNEsSingle)
    }
    
    // Counts Named Entities
    def countNEs(result: Seq[Seq[String]]): Map[String, Int] = {
        result.flatten.foldLeft(Map.empty[String, Int]) {
         (count, word) => count + (word -> (count.getOrElse(word, 0) + 1)) }
    }
    
    // Sorts Named Entities
    def sortNEs(counts: Map[String, Int]): List[(String, Int)] = {
        counts.toList.sortBy(_._2)(Ordering[Int].reverse)
    } 
}

defined class NERModel

In [25]:
val model = new NERModel(STOPWORDS)

model: NERModel = ammonite.$sess.cmd23$Helper$NERModel@4f88f068

### 2.2 Aplicar el "Modelo" a los datos

In [26]:
val result = model.getNEs(redditText)

result: Seq[Seq[String]] = List(
  ArrayBuffer(
    "Moronic",
    "Monday",
    "May",
    "Note",
    "Join",
    "Ctrl-F",
    "Note",
    "Join",
    "IRC",
    "Telegram"
  ),
  ArrayBuffer("Google", "Assistant", "Android"),
  ArrayBuffer(
    "Arm",
    "Announces",
    "Mobile",
    "Armv9",
    "CPU",
    "Microarchitectures",
    "Cortex-X2",
    "Cortex-A710",
    "Cortex-A510"
  ),
  ArrayBuffer("Anker", "Nebula", "Android", "TV"),
  ArrayBuffer(
    "Galaxy",
    "Upcycling",
    "Samsung",
    "Ruined",
    "Best",
    "Idea",
    "Years"
  ),
  ArrayBuffer("VideoCardz", "Mali-G710", "G610", "G510", "G310"),
  ArrayBuffer("Android", "Beta"),
  ArrayBuffer("Google", "Photos"),
  ArrayBuffer("Honor\u2019s", "Google"),
...

## 3. Contar y ordenar las entidades

Concatenar todas las listas, contar cada Named Entity, y luego ordernar por frecuencia

In [57]:
val counts = model.countNEs(result)
val sortedNEs = model.sortNEs(counts)

counts: Map[String, Int] = Map(
  "Thing" -> 1,
  "Advertising" -> 1,
  "PLN" -> 5,
  "Please" -> 1,
  "Easy" -> 1,
  "Locked" -> 1,
  "One" -> 8,
  "CEO" -> 1,
  "Tab" -> 3,
  "Weinbach" -> 3,
  "Use" -> 1,
  "Cheaper" -> 1,
  "Take" -> 1,
  "OLED" -> 1,
  "NPUs" -> 1,
  "A11" -> 3,
  "Disagree" -> 1,
  "Upcycling" -> 1,
  "ISP" -> 1,
  "Settings" -> 1,
  "Lite" -> 1,
  "Pichai" -> 1,
  "Users" -> 1,
  "Reddit" -> 1,
  "GPS" -> 1,
  "Google/Samsung/HTC" -> 1,
  "UI" -> 5,
  "Added" -> 1,
  "Ctrl-F" -> 2,
  "Folder" -> 1,
  "ROMs" -> 1,
  "YouTube" -> 1,
  "Galaxy" -> 5,
  "Honor\u2019s" -> 1,
  "Note" -> 5,
  "Dual" -> 1,
  "Xiaomi" -> 2,
  "Best" -> 1,
...
sortedNEs: List[(String, Int)] = List(
  ("Pixel", 20),
  ("Google", 20),
  ("Android", 17),
  ("Apple", 14),
  ("One", 8),
  ("Samsung", 6),
  ("PLN", 5),
  ("UI", 5),
  ("Galaxy", 5),
  ("Note", 5),
  ("App", 5),
  ("Join", 5),
  ("USD", 5),
  ("Max", 4),
  ("Photos", 4),
  ("Bluetooth", 4),
  ("Tab", 3),
  ("Weinbach", 3),
  ("A